In [1]:
import re
import pickle
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd

### 1. Get Current Fantasy Points

In [2]:
pfr_fantasy_link = "https://www.pro-football-reference.com/years/2022/fantasy.htm"

def getCurrentFantasyPoints(link):
    info_dict = {'born' : None,
                    'draft': None,
                    'pos' : None}

    page = requests.get(link)
    if page.status_code == 200:
        soup = BeautifulSoup(page.content, 'html.parser')
    player_info  = soup.find_all('p')
    
    table = soup.find_all('table', id = 'fantasy')
    df = pd.read_html(str(table), flavor = 'html5lib')[0]
    df.columns = df.columns.get_level_values(1)
    df = df[df['Player'] != 'Player'].copy()
    df['PPR'] = pd.to_numeric(df['PPR'])
    df['PPR'].fillna(0, inplace= True)

    df.loc[df['Player'].str.contains('Etienne'), 'Player'] = 'Travis Etienne Jr.'
    return df[df['PPR'] > 10].copy()

currPts = getCurrentFantasyPoints(pfr_fantasy_link)
currPts = currPts[['Player','Tm','FantPos','G','PPR']]
currPts

,Player,Tm,FantPos,G,PPR
0,Derrick Henry,TEN,RB,8,169.0
1,Josh Allen,BUF,QB,8,215.3
2,Nick Chubb,CLE,RB,8,163.1
3,Travis Kelce,KAN,TE,8,167.4
4,Patrick Mahomes,KAN,QB,8,203.8
...,...,...,...,...,...
383,Quintin Morris,BUF,TE,7,10.7
384,Albert Okwuegbunam,DEN,TE,5,12.0
386,James Proche,BAL,WR,7,11.3
390,Mitchell Wilcox,CIN,TE,9,10.7


### 2. Get Projections

##### 2a. My Projections

In [52]:
with open('../projections/player_proj_2022.p', 'rb') as handle:
    proj = pickle.load(handle)
proj = proj[['Player','Tm','FantPos','PrvPts_PPR','AverageDraftPositionPPR','Preds','Preds_adp','Prob']]

##### 2b. ESPN Projections

In [53]:
# Games played dictionary
page = requests.get("https://www.pro-football-reference.com/")
if page.status_code == 200:
    soup = BeautifulSoup(page.content, 'html.parser')

games_played = {'2TM' : 9}
afc = soup.find('table', id = 'AFC')
nfc  = soup.find('table', id = 'NFC')

for conference in ['AFC', 'NFC']:    
    conf = soup.find('table', id = conference)
    for row in conf.find_all('tr'):
        head = row.find_all('th', {'scope' : 'row'})
        if len(head) > 0:
            tm_name = re.sub('[^a-zA-Z]+', '', row.find('th').text)
            cols = row.find_all('td')
            gp = int(cols[0].text) + int(cols[1].text) + int(cols[2].text)
            games_played[tm_name] = gp
print(games_played)

{'2TM': 9, 'BUF': 8, 'NYJ': 9, 'MIA': 9, 'NWE': 9, 'BAL': 9, 'CIN': 9, 'CLE': 8, 'PIT': 8, 'TEN': 8, 'IND': 9, 'JAX': 9, 'HOU': 8, 'KAN': 8, 'LAC': 8, 'DEN': 8, 'LVR': 8, 'PHI': 8, 'DAL': 8, 'NYG': 8, 'WAS': 9, 'MIN': 8, 'GNB': 9, 'CHI': 9, 'DET': 8, 'TAM': 9, 'ATL': 9, 'NOR': 9, 'CAR': 9, 'SEA': 9, 'SFO': 8, 'LAR': 8, 'ARI': 9}


In [54]:
espn_to_pfr_dict = {'Ari':'ARI',
                    'Atl': 'ATL',
                    'Bal': 'BAL',
                    'Buf': 'BUF',
                    'Car': 'CAR',
                    'Chi': 'CHI',
                    'Cin': 'CIN',
                    'Cle': 'CLE',
                    'Dal': 'DAL',
                    'Den': 'DEN',
                    'Det': 'DET',
                    'GB': 'GNB',
                    'Hou': 'HOU',
                    'Ind': 'IND',
                    'Jax': 'JAX',
                    'KC': 'KAN',
                    'LAC': 'LAC',
                    'LAR': 'LAR',
                    'LV': 'LVR',
                    'Mia': 'MIA',
                    'Min': 'MIN',
                    'NE': 'NWE',
                    'NO': 'NOR',
                    'NYG': 'NYG',
                    'NYJ': 'NYJ',
                    'Phi': 'PHI',
                    'Pit': 'PIT',
                    'Sea': 'SEA',
                    'SF': 'SFO',
                    'TB': 'TAM',
                    'Ten': 'TEN',
                    'Wsh': 'WAS'}


In [56]:
def getESPNprojs(link = '../projections/projections_espn.xlsx'):
    proj_espn = pd.read_excel(link, sheet_name = 'projections', usecols = "A:E")
    proj_espn['Tm'] = proj_espn['Tm'].replace(espn_to_pfr_dict)
    proj_espn.drop('TeamPosition', axis = 1, inplace = True)
    proj_espn['Points'] = pd.to_numeric(proj_espn['Points'], errors = 'coerce')
    proj_espn['Points'].fillna(0)
    return proj_espn

proj_espn = getESPNprojs()

In [58]:
def mergeProjections(currPts, proj, proj_espn):
    """ 1. Merge on my projections """
    newPts = currPts.merge(proj, on = ['Player', 'Tm', 'FantPos'], how = 'left', indicator = True)
    # print(newPts.loc[newPts['_merge']!='both'].sort_values('PPR',ascending = False))
    newPts.loc[newPts['_merge']!='both', 'Preds'] = 0

    """ 2. Merge on ESPN projections"""
    newerPts = newPts.merge(proj_espn, left_on= ['Player','Tm','FantPos'], right_on = ['Player','Tm','Position'],  how='left', indicator = 'espn')
    # print(newerPts['espn'].value_counts())
    newerPts[newerPts['espn'] == 'left_only'].sort_values('PPR', ascending = False)

    """ 3. Mark guys without projections"""
    newerPts.loc[newerPts['_merge']!= 'both', 'Preds'] = 10
    newerPts.loc[newerPts['espn']!= 'both', 'Points'] = 10
    
    """ 4. Prorate PPR metric so far"""
    newerPts['GP'] = pd.to_numeric(newerPts['Tm'].replace(games_played))
    newerPts['PPR_adj'] = newerPts['PPR'] * newerPts['GP']

    newerPts.rename(columns = {'Points' : 'Points_ESPN'}, inplace = True)
    return newerPts
newerPts = mergeProjections(currPts, proj, proj_espn)
newerPts
newerPts.to_excel('test.xlsx')

,Player,Tm,FantPos,G,PPR,PrvPts_PPR,AverageDraftPositionPPR,Preds,Preds_adp,Prob,_merge,Position,Points_ESPN,espn,GP,PPR_adj
0,Derrick Henry,TEN,RB,8,169.0,193.30,4.3,217.964970,216.077163,0.664468,both,RB,264.5,both,8,1352.0
1,Josh Allen,BUF,QB,8,215.3,402.58,20.4,321.917298,317.955804,0.799259,both,QB,374.6,both,8,1722.4
2,Nick Chubb,CLE,RB,8,163.1,215.30,18.8,189.985782,198.812688,0.518539,both,RB,216.0,both,8,1304.8
3,Travis Kelce,KAN,TE,8,167.4,262.80,16.5,198.122198,200.481549,0.752541,both,TE,227.3,both,8,1339.2
4,Patrick Mahomes,KAN,QB,8,203.8,361.66,38.3,315.149123,299.704937,0.783244,both,QB,352.0,both,8,1630.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,Quintin Morris,BUF,TE,7,10.7,NaN,NaN,10.000000,NaN,NaN,left_only,TE,4.9,both,8,85.6
348,Albert Okwuegbunam,DEN,TE,5,12.0,76.00,173.0,74.374224,25.176465,0.062416,both,TE,122.5,both,8,96.0
349,James Proche,BAL,WR,7,11.3,36.20,173.0,58.293357,28.664191,0.030584,both,NaN,10.0,left_only,9,101.7
350,Mitchell Wilcox,CIN,TE,9,10.7,4.60,173.0,25.440222,25.176465,0.009270,both,TE,9.1,both,9,96.3


In [48]:
newerPts['Points_ESPN'].isnull().sum()

0

In [50]:
from sklearn.metrics import r2_score

print(newerPts['Preds'].corr(newerPts['PPR_adj'])**2)
print(newerPts['Points_ESPN'].corr(newerPts['PPR_adj'])**2)
print(newerPts['Preds_adp'].corr(newerPts['PPR_adj'])**2)

newerPts['PPR_adj']

0.5142843531969153
0.4898462728516379
0.4809070592515135


0      1352.0
1      1722.4
2      1304.8
3      1339.2
4      1630.4
        ...  
347      85.6
348      96.0
349     101.7
350      96.3
351      94.4
Name: PPR_adj, Length: 352, dtype: float64